Importing Library and api_keys check

In [2]:
# Import environment loading library
from dotenv import load_dotenv
import json
# Import IBMGen Library 
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from langchain.llms.base import LLM
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
# Import lang Chain Interface object
# from langChainInterface import LangChainInterface
# Import system libraries
import os
# Import streamlit for the UI 
import streamlit as st
import re
from ibm_watson_machine_learning.foundation_models import Model
import docx
import pandas as pd

# Load environment vars
load_dotenv()

# Define credentials 
api_key = os.getenv("API_KEY", None)
ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)
project_id = os.getenv("PROJECT_ID", None)
serp_api_key = os.getenv("SERP_API_KEY",None)


if api_key is None or ibm_cloud_url is None or project_id is None:
    print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }


Forcasting Revenue 

In [3]:
df_all = pd.read_csv("data_sum_per_region_date_all_dept.csv")
df_new = df_all[["Department", "Revenue", "YearMonth"]]
df_new["YearMonth"] = pd.to_datetime(df_new["YearMonth"])
df_2022_11 = df_new[df_new["YearMonth"] <= "2022-11"]

C:\Users\BrianS\AppData\Local\Temp\ipykernel_13436\2164439683.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["YearMonth"] = pd.to_datetime(df_new["YearMonth"])


In [6]:
def revenue_generator(df,i, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 100,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            "stop_sequences": ["\n\n"]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"{df} is a dataset of a company.\n"\
                "Predict the next one month revenue based on the data provided.\n"\
                "Please give the answear only the value of the prediction.\n"\
                "[/INST]\n"\
                "Output:"
                

        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response


Prediction With Checking (Predicting revenue value 2022)

In [8]:
dep_name = df_2022_11['Department'].unique().tolist()
df_per_team = pd.DataFrame()
result_list = []
for i in dep_name:
    print(f'Process Running for Departement{i}')
    df_per_team = df_2022_11[df_2022_11['Department']==i]
    result = revenue_generator(df_per_team, i)
    result_json = {i:result}
    print(result_json)
    result_list.append(result_json)
    
    print(f'Process Complete for Departement {i}')
    print("prediction has been completed.")

print(result_list)    

Process Running for DepartementAccount Management
{'Account Management': ' 52845.89'}
Process Complete for Departement Account Management
prediction has been completed.
Process Running for DepartementBusiness Development
{'Business Development': ' 54250.89'}
Process Complete for Departement Business Development
prediction has been completed.
Process Running for DepartementChannel Sales
{'Channel Sales': ' 25,000.00'}
Process Complete for Departement Channel Sales
prediction has been completed.
Process Running for DepartementCustomer Success
{'Customer Success': ' 55642.89'}
Process Complete for Departement Customer Success
prediction has been completed.
Process Running for DepartementE-commerce Sales
{'E-commerce Sales': ' 75000\n\n'}
Process Complete for Departement E-commerce Sales
prediction has been completed.
Process Running for DepartementEnterprise Sales
{'Enterprise Sales': ' 59878.92'}
Process Complete for Departement Enterprise Sales
prediction has been completed.
Process Run

Last Month Real value for comparing with the revenue predicted values

In [9]:
df_2022_12 = df_new[df_new["YearMonth"] == "2022-12"]
print(df_2022_12)

                  Department    Revenue  YearMonth
71        Account Management   58283.11 2022-12-01
143     Business Development   52286.02 2022-12-01
215            Channel Sales   30134.25 2022-12-01
287         Customer Success   71357.59 2022-12-01
359         E-commerce Sales  104492.57 2022-12-01
431         Enterprise Sales   69478.81 2022-12-01
503             Inside Sales   91318.56 2022-12-01
575      International Sales   63081.57 2022-12-01
647             Key Accounts  113740.07 2022-12-01
719            Outside Sales   83995.40 2022-12-01
791             Retail Sales  100521.87 2022-12-01
863     Sales Administration   24037.29 2022-12-01
935          Sales Analytics  110008.72 2022-12-01
1007        Sales Enablement   30265.16 2022-12-01
1079       Sales Engineering   25524.96 2022-12-01
1151        Sales Operations   94512.04 2022-12-01
1223           Sales Support   93343.62 2022-12-01
1295          Sales Training  116266.57 2022-12-01
1367  Strategic Partnerships   

Revenue Growth from the last revenue.

In [ ]:
pred_data = {
  "revenue_prediction": [
    {
      "Retail Sales": "102784.99",
      "Sales Administration": "24678.92",
      "Account Management": "52845.89",
      "Business Development": "54250.89",
      "Channel Sales": "25000.00",
      "Customer Success": "55642.89",
      "E-commerce Sales": "75000",
      "Enterprise Sales": "59878.92",
      "Inside Sales": "72342.89",
      "International Sales": "58924.87",
      "Key Accounts": "78234.52",
      "Outside Sales": "68421.32",
      "Retail Sales": "69544.89",
      "Sales Administration": "24678.92",
      "Sales Analytics": "78943.89",
      "Sales Enablement": "23542.15",
      "Sales Engineering": "24567.89",
      "Sales Operations": "65425.89",
      "Sales Support": "98745.26",
      "Sales Training": "89421.25",
      "Strategic Partnerships": "85000"
    }
  ]
}


In [ ]:
df_only_2022_11 = df_new[df_new["YearMonth"] == "2022-11"]
df_only_2022_11.tail(20)



,Department,Revenue,YearMonth
70,Account Management,64799.64,2022-11-01
142,Business Development,61950.15,2022-11-01
214,Channel Sales,27661.48,2022-11-01
286,Customer Success,58988.97,2022-11-01
358,E-commerce Sales,77430.95,2022-11-01
430,Enterprise Sales,57580.44,2022-11-01
502,Inside Sales,84916.04,2022-11-01
574,International Sales,53415.52,2022-11-01
646,Key Accounts,75239.96,2022-11-01
718,Outside Sales,67310.19,2022-11-01


In [ ]:
last_revenue = {
  "real_revenue": {
    "Account Management": 64799.64,
    "Business Development": 61950.15,
    "Channel Sales": 27661.48,
    "Customer Success": 58988.97,
    "E-commerce Sales": 77430.95,
    "Enterprise Sales": 57580.44,
    "Inside Sales": 84916.04,
    "International Sales": 53415.52,
    "Key Accounts": 75239.96,
    "Outside Sales": 67310.19,
    "Retail Sales": 67396.28,
    "Sales Administration": 21590.11,
    "Sales Analytics": 74278.92,
    "Sales Enablement": 21327.32,
    "Sales Engineering": 23452.96,
    "Sales Operations": 63181.58,
    "Sales Support": 97633.48,
    "Sales Training": 87132.42,
    "Strategic Partnerships": 82990.05
  }
}



In [ ]:
def rev_growth(real,pred, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 1000,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            #"Temperature" : 0.7
            # "stop_sequences": [""]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"{last_revenue} is the last revenue data of a company.\n"\
                f"{pred_data} is a prediction of a company revenue for the coming month. \n"\
                f"Calculate the revenue growth from {last_revenue} with {pred_data}.\n"\
                "[/INST]\n"\
                "Output:"

        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response


In [ ]:
rev_growth_result = rev_growth(last_revenue,pred_data)
print(rev_growth_result)



The revenue growth from the current revenue data to the predicted revenue data is:

* Retail Sales: 69544.89 - 67396.28 = 2148.61 (3.19%)
* Sales Administration: 24678.92 - 21590.11 = 3088.81 (14.3%)
* Account Management: 52845.89 - 64799.64 = -11953.75 (-18.5%)
* Business Development: 54250.89 - 61950.15 = -7700.06 (-12.5%)
* Channel Sales: 25000.00 - 27661.48 = -2661.48 (-9.6%)
* Customer Success: 55642.89 - 58988.97 = -3346.08 (-5.7%)
* E-commerce Sales: 75000 - 77430.95 = -2430.95 (-3.2%)
* Enterprise Sales: 59878.92 - 57580.44 = 2298.48 (4.0%)
* Inside Sales: 72342.89 - 84916.04 = -12573.15 (-14.8%)
* International Sales: 58924.87 - 53415.52 = 5509.35 (10.3%)
* Key Accounts: 78234.52 - 75239.96 = 2994.56 (4.0%)
* Outside Sales: 68421.32 - 67310.19 = 1111.13 (1.7%)
* Sales Analytics: 78943.89 - 74278.92 = 4664.97 (6.3%)
* Sales Enablement: 23542.15 - 21327.32 = 2214.83 (10.4%)
* Sales Engineering: 24567.89 - 23452.96 = 1114.93 (4.8%)
* Sales Operations: 65425.89 - 63181.58 = 2244

Evaluating and analyzing job responsibility with goal

In [4]:

goal= {
    "company_goals": ["Increasing profit margin and keep increasing transaction users"]
  }



Finance

In [20]:
job_finance =  [
    {
        "department": "Finance Director",
        "JobResponsibilities": [
            "Supervise the business and budget of the Company and business units to ensure that performance is in accordance with the set budget.",
            "Establish budget assumptions and business units as well as budget preparation guidelines to ensure that the budget prepared is realistic.",
            "Directs the preparation of all financial statements, including income statements, balance sheets, shareholder reports, tax returns, and governmental agency reports.",
            "Create and establish yearly financial objectives that align with the company's plan for growth and expansion.",
            "Compares sales and profit projections to actual figures and budgeted expenses to actual expenses; makes or oversees any necessary adjustments to future projections and budgets.",
            "Completing tax rights and obligations in accordance with applicable regulations and the implementation of Good Corporate Governance principles.",
            "Ensure that the Company's accounting reporting process complies with applicable regulations.",
            "Coordinating legal advice, ensuring compliance with legislation etc.",
            "Works with the CEO and other executives to coordinate planning and establish priorities for the planning process.",
            "Lead employees to encourage maximum performance and dedication"
        ]
    },
    {
        "department": "Finance",
        "JobResponsibilities": [
            "Establish budget assumptions and business units as well as budget preparation guidelines to ensure that the budget prepared is realistic.",
            "Create and establish yearly financial objectives that align with the company's plan for growth and expansion.",
            "Compares sales and profit projections to actual figures and budgeted expenses to actual expenses; makes or oversees any necessary adjustments to future projections and budgets.",
            "Completing tax rights and obligations in accordance with applicable regulations and the implementation of Good Corporate Governance principles.",
            "Manage financial planning and analysis department. Supervise creation of reports, software implementation and tools for budgeting and forecasting.",
            "Prepare and present monthly financial budgeting reports including monthly profit and loss by division, forecast vs. budget and cash flow.",
            "Mentor and develop team, managing work allocation, systems training, performance evaluations, and the building of an effective and efficient team dynamic.",
            "Ensure that the Company's accounting reporting process complies with applicable regulations.",
            "Lead employees to encourage maximum performance and dedication."
        ]
    },
    {
        "department": "Legal",
        "JobResponsibilities": [
            "Provide legal counsels and services for operational and business process and company management",
            "Review and establish legal materials and regulations to protect company's business and interest",
            "Provide legal services for acquisition process, joint venture, asset managements, and subsidiary's acquisition or liquidation process.",
            "Manage and supervise external consultants hired to provide advises and guidance",
            "Manage and monitor legal disputes and claims involving business and operational aspects",
            "Review, establish, and manage systems and procedures to enhance efficiency and effectiveness of internal legal process",
            "Review, distribute, and socialize legal aspects with related internal units.",
            "Responsible and ensure accelerating and effectiveness of corporate project initiatives, including Identifying and resolving the obstacles, Establishing and maintaining SLA of project completeness, gathering commitment with any involved / related units,",
            "Manage tracking and reporting of project status and provide project performance and recommend the improvement analysis",
            "Identifying, formulating and recommend other initiatives to improve project result.",
            "Leads, directs, evaluates, and develops teams to ensure the achievement of organizational goals.",
            "Monitors and evaluates the performance of group and individual staff, recommending and implementing personnel actions, such as promotions and dismissals."
        ]
    }
]

Finance Model 

In [22]:
def finance_optimized_org(input_data, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 3000,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            #"Temperature" : 0.7
            # "stop_sequences": [""]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"generate the response with {goal} as consideration"\
                f"check in this data {job_finance} is there any job responsibility in one department that is same with any other department.\n"\
                "If there is job responsibility in one department that is same with any other department. generate a recommendation to make the job responsibility distinct and effective."\
                "Make the recommendation per point in department that has same jobdesk and modified version of it."\
                "if it needed generate which job desk that need to cut or merge because of the job desk to differentiate and the reason behind it."\
                "Generate the output per point with department and recommended job responsibilities as keys."\
                "do not generate again the input job responsibility as an output."\
                "do not generate the answear in json."\
                "[/INST]\n"\
                "Output:"
 #make a few shot
        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response


In [23]:
ans_finance = finance_optimized_org(job_finance)
print(ans_finance)



1. Department: Finance
Recommended Job Responsibilities:
* Establish budget assumptions and business units as well as budget preparation guidelines to ensure that the budget prepared is realistic.
* Create and establish yearly financial objectives that align with the company's plan for growth and expansion.
* Compares sales and profit projections to actual figures and budgeted expenses to actual expenses; makes or oversees any necessary adjustments to future projections and budgets.
* Completing tax rights and obligations in accordance with applicable regulations and the implementation of Good Corporate Governance principles.
* Manage financial planning and analysis department. Supervise creation of reports, software implementation and tools for budgeting and forecasting.
* Prepare and present monthly financial budgeting reports including monthly profit and loss by division, forecast vs. budget and cash flow.
* Mentor and develop team, managing work allocation, systems training, perf

Operation jobs

In [77]:
job_operation = [
    {
        "department": "Customer Management",
        "responsibilities": [
            "Develop and implement a comprehensive strategy that aligns with the organization's goals and objectives.",
            "Lead and manage the team, including customer service, account management, and customer success functions.",
            "Establish and maintain strong relationships with key customers, understanding their needs, and acting as a trusted advisor.",
            "Develop and implement customer retention and loyalty programs to maximize customer satisfaction and minimize churn.",
            "Monitor and analyze customer data, feedback, and market trends to identify opportunities for improvement and develop actionable insights.",
            "Collaborate with cross-functional teams, including sales, marketing, and product development, to ensure a customer-centric approach across the organization.",
            "Drive initiatives to enhance the overall customer experience, including process improvements, service enhancements, and personalized offerings.",
            "Develop and monitor key performance indicators (KPIs) related to customer satisfaction, retention, and revenue growth.",
            "Implement customer feedback mechanisms, such as surveys and focus groups, to gather insights and drive continuous improvement.",
            "Stay updated with industry trends, emerging technologies, and best practices in customer management, bringing innovative ideas and solutions to the organization.",
            "Provide strategic guidance to senior management on customer-related matters, including market trends, competitive intelligence, and customer needs.",
            "Collaborate with the sales team to identify upselling and cross-selling opportunities, driving revenue growth from existing customers.",
            "Oversee the resolution of customer escalations and ensure timely and effective response to customer inquiries or issues.",
            "Develop and manage budgets, ensuring efficient resource allocation and cost control."
        ]
    },
    {
        "department": "Human Capital",
        "responsibilities": [
            "Develops human resource planning models to identify competency, knowledge and talent gaps and develop specific programs to fill the identified gaps. Areas of activity will include talent management through proper succession planning programs for key contributor and management positions, training, and development programs for preparing employees for more significant responsibilities, and general business development programs to enhance employee knowledge and understanding of the business of the company.",
            "Enhances and/or develops, implements, and enforces HR policies and procedures of the organization by way of systems that will improve the overall operation and effectiveness of the organization. Manages the human resource information systems database and provides necessary reports for critical analyses of the HR function and the people resources of the organization.",
            "Participates in the development of the organization's plans and programs as a strategic partner, but particularly from the perspective of the impact on people.",
            "Translates the strategic and tactical business plans into HR strategic and operational plans.",
            "Evaluates and advises on the impact of long-range planning of new programs/strategies and regulatory action as those items affect the attraction, motivation, development, and retention of the people resources of the organization.",
            "Develops appropriate policies and programs for effective management of the people resources of the organization",
            "Develops progressive and proactive compensation and benefits programs to provide motivation, incentives, and rewards for effective performance.",
            "Develops staffing strategies and implementation plans and programs to identify talent within and outside the organization for positions of responsibility. Identifies appropriate and effective external sources for candidates for all levels within the organization.",
            "Continues improving the programs, policies, practices, and processes associated with meeting the strategic and operational people issue of the organization.",
            "Continually assesses the competitiveness of all programs and practices against the relevant comparable companies, industries, and markets."
        ]
    },
    {
        "department": "Operations Excellence",
        "responsibilities": [
            "Acts as a liaison with Management to ensure a full understanding of business strategies, planning, execution, performance, and risks at the business unit level",
            "Lead operational strategic planning & performance measurement, including drafting, executing and maintaining the Operational Plan itself, measuring project performance over time against the Plan's targets",
            "Contribute to strategic leadership, including managing Operations's self-reflection efforts to ensure that the organization is continuously iterating, learning, and improving",
            "Oversee resource allocation and implementation of any initiatives and projects to bring added value to the company.",
            "Leading key relationships with Strategic partners to develop business Strategic alliance",
            "Periodically mentor and coach with each subordinate to discuss the progress towards achieving the goals and assessing their performance, evaluating each target area, behavior, knowledge, and skills",
            "Set prioritization, determine assignments and resources, monitor the results of delegation, assignment, or project, by considering the skills, knowledge, and experience of everyone in the assigned team, as well as the characteristics of the project",
            "Outstanding analytical, communication, and organizational skills to inspire team for problem solving",
            "Ensure Strategic Operations unit framework run effectively and efficiently by utilizing a system or certain techniques to monitor the performance",
            "Determine and monitor budget, resources, and costs of project implementations",
            "Responsible for successful completion of tasks with high performance standards for both individual and team.",
            "Act as the Agent of Change, which always drives improvements to provide best customer experience & satisfaction.",
            "Other responsibilities as assigned"
        ]
    },
    {
        "department": "Premium Care",
        "responsibilities": [
            "Proactively contacting top sales segmented sellers that have experience in contacting Seller Care contact center to help them boost their exposure and sales as well as gathering their feedback.",
            "Conducting sellers' training as well as maintaining seller engagement by phone call, WhatsApp and visit for better sell-on-abc literacy to boost seller growth.",
            "Utilizing Premium Seller Care empowerments such as: AHA Moment, Seller Visit, Prioritized Case Resolution, and other empowerments to increase seller satisfaction and performance.",
            "Retaining sellers who churned due to operational cases, especially return cases by using various empowerments for more seamless seller experience.",
            "Gathering sellers' insights about their journey and conducting cross function data analysis for future improvements.",
            "Monitoring sellers' performance, proactively contacting them if they find possible issues that might arise and solving them in a timely manner."
        ]
    },
    {
        "department": "Supply Chain",
        "responsibilities": [
            "Develop and implement the overall operational strategy for logistics and warehouse management, aligning it with the organization's goals and objectives.",
            "Provide leadership and direction to the operations team, including logistics managers, warehouse supervisors, and other personnel, ensuring effective coordination and collaboration.",
            "Optimize logistics and warehouse processes to ensure efficient transportation, inventory management, storage, and distribution of goods.",
            "Monitor and analyze key performance indicators (KPIs) such as on-time delivery, order fulfillment, inventory accuracy, and warehouse productivity.",
            "Identify areas for improvement and implement strategies to enhance operational efficiency, reduce costs, and streamline processes.",
            "Collaborate with cross-functional teams, including procurement, sales, and customer service, to align operational activities and meet customer demands.",
            "Oversee and manage relationships with logistics service providers, carriers, and third-party logistics (3PL) partners to ensure seamless operations and adherence to service level agreements.",
            "Ensure compliance with regulatory requirements, safety standards, and industry best practices related to logistics and warehouse operations.",
            "Develop and monitor operational budgets, forecast resource requirements, and make strategic decisions to optimize resource allocation.",
            "Stay informed about technological advancements and industry trends in logistics and warehouse management, identifying opportunities for innovation and improvement.",
            "Foster a culture of continuous improvement, quality control, and operational excellence throughout the logistics and warehouse departments.",
            "Provide regular reports and updates to senior management on operational performance, challenges, and improvement initiatives."
        ]
    }
]

In [102]:
def operation_optimized_org(input_data, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 4050,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            #"Temperature" : 0.7
            # "stop_sequences": [""]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"generate the response with {goal} as consideration"\
                f"check in this data {job_operation} is there any job responsibility in one department that is same with any other department.\n"\
                "If there is job responsibility in one department that is same with any other department. generate a recommendation to make the job responsibility distinct and effective."\
                "Make the recommendation per point in department that has same jobdesk and modified version of it."\
                "if it needed generate which job desk that need to cut or merge because of the job desk to differentiate and the reason behind it."\
                "Generate the output per point with department and recommended job responsibilities as keys."\
                "do not generate again the input job responsibility as an output."\
                "do not generate the answear in json."\
                "[/INST]\n"\
                "Output:"
 #make a few shot
        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response


In [103]:
ans_operation = operation_optimized_org(job_operation)
print(ans_operation)



1. Department: Customer Management
Recommended Job Responsibilities:
* Develop and implement a comprehensive customer strategy that aligns with the organization's goals and objectives.
* Lead and manage the customer service, account management, and customer success teams.
* Establish and maintain strong relationships with key customers, understanding their needs, and acting as a trusted advisor.
* Develop and implement customer retention and loyalty programs to maximize customer satisfaction and minimize churn.
* Monitor and analyze customer data, feedback, and market trends to identify opportunities for improvement and develop actionable insights.
* Collaborate with cross-functional teams, including sales, marketing, and product development, to ensure a customer-centric approach across the organization.
* Drive initiatives to enhance the overall customer experience, including process improvements, service enhancements, and personalized offerings.
* Develop and monitor key performanc

In [108]:
job_tech = [
    {
        "department": "Data",
        "JobResponsibilities": [
            "Deliver Predictive Analytics, Data Mining and Machine Learning driven solutions",
            "Identify, analyse, and interpret trends or patterns in complex data sets",
            "Build complex data sets from multiple data sources, both internally and externally.",
            "Build machine learning models to solve business/product problems or requirements in ecommerce and digital marketing domains",
            "Understand business use cases, create appropriate training and test data sets from multiple data sources, carry out model validation exercises and fine tune models for accuracy",
            "Use visualization techniques and tools to present actionable insights to business users",
            "Keep up-to-date with latest technology trends",
            "Mentoring junior team members on technical subjects",
            "Working in a team environment, interact with multiple groups on a daily basis",
            "Recruit staff to include interviewing, hiring, assigning work, training, coaching and mentoring",
            "Conducts performance review and holds staff accountable for performance of their responsibility",
            "Conducts weekly/monthly one on one as forum to connect with the team member",
            "Maintain a good relationship with other senior management",
            "Plan & manage roadmap for Data Projects"
        ]
    },
    {
        "department": "Developer",
        "JobResponsibilities": [
            "Manage the prioritization and work assignment among different development groups/development leads",
            "Act as a central point of communication between the team and the stakeholders of other department: Product Management, Program and Project Management",
            "Identify, manage and mitigate risk throughout the project lifecycle and keep stakeholders aware of risks/progress",
            "Coach, motivate, lead and supervise software development team members",
            "Work with the team to improve development processes, policies, tools and practices to ensure they are in alignment with the department priorities and industry best practices",
            "Recruit staff to include interviewing, hiring, assigning work, training, coaching and mentoring",
            "Conducts performance review and holds staff accountable for performance of their responsibility",
            "Conducts weekly/monthly one on one as forum to connect with team members",
            "Have some business skills specifically in the area of e-commerce for Indonesia market",
            "Able to describe technology in simple form where non technical person can understand",
            "Proactively seek solution to a problem (either internally found or customer reported issue)",
            "Create creative solution either on standalone system or overall system",
            "Improve operations by conducting systems analysis; recommending changes in policies and procedures",
            "Updates job knowledge by studying state of the art development tools, programming techniques and computing equipment; participating in educational opportunities; reading professional publications; maintaining personal networks; participating in professional organizations",
            "Protect operations by keeping information confidential",
            "Maintain excellent coding practices, verbal and written communication skills"
        ]
    },
    {
        "department": "Head of Infrastructure & Technical Support",
        "JobResponsibilities": [
            "Responsible for the creation, design, management, monitoring, and support of the IT infrastructure (networks, database and server) and services, supporting architecture and roadmaps to ensure the selection, deployment and lifecycle management of appropriate services for the organization.",
            "Develop and implement IT policies and architectural and design standards with respect to network, compute, storage, and cloud services.",
            "Develop enterprise standards and technology architecture and the IT operations governance process.",
            "Manages IT infrastructure work and resource demands in an orderly manner, supplying resources for new projects and work requests, and ensuring that solutions proposed are aligned with the applications strategy.",
            "Resolve complex technical issues and provide senior technical leadership in response to queries from across the business as well as from within IT.",
            "Participate in the formulation of the company's enterprise architecture and business system plans; assessing cost and feasibility, and ensuring the plan is aligned with and supports the strategic goals of the business.",
            "Manages the performance objectives and development team including the monitoring, review and validation of individual training and development plans in line with organizational and business requirements.",
            "Lead, managed and develop team to maximize productivity, ensure the skills pool is current and relevant to the requirements of the organization, enhance job satisfaction, promote career development and implement effective succession planning."
        ]
    },
    {
        "department": "Software Quality",
        "JobResponsibilities": [
            "Recruit staff to include interviewing, hiring, assigning work, training, coaching and mentoring",
            "Conducts performance review and holds staff accountable for performance of their responsibility",
            "Conducts weekly/monthly one on one as forum to connect with the team member",
            "Have some business skills specifically in the area of e-commerce for Indonesia market",
            "Manage, prioritize and execute multiple concurrent software testing activities",
            "Identify, manage and mitigate risk throughout the project lifecycle and keep stakeholders aware of risks/progress",
            "Subject matter expert, go to person of the field",
            "Able to describe how the testing procedure is being done in simple form where a non-technical person can understand",
            "Proactively seek solution by reproducing a problem (either internally found or customer reported issue)",
            "Create creative solution to any problem preventing automation test",
            "Train and mentor to SDET (Software Development Engineer and Test)",
            "Participate in measuring and driving product quality from inception to release",
            "Participate in measuring system performance and communicate if there is a drop on performance",
            "Communication with the system analyst on the requirement",
            "Assessment to user experience and usability",
            "Responsible for the quality of the selected product features",
            "Design and develop manual or automated test systems to validate functionality, performance, stability and security of past and current features",
            "Updates job knowledge by studying state of the art development tools, programming techniques and computing equipment; participating in educational opportunities; reading professional publications; maintaining personal networks; participating in professional organizations",
            "Protect operations by keeping information confidential",
            "Accomplishes engineer and organization mission by completing related results as needed",
            "Maintain discipline of written report which captures the week's accomplishments and issues"
        ]
    }
]

In [113]:
def tech_optimized_org(input_data, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 3000,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            #"Temperature" : 0.7
            # "stop_sequences": [""]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"generate the response with {goal} as consideration"\
                f"check in this data {job_tech} is there any job responsibility in one department that is same with any other department.\n"\
                "If there is job responsibility in one department that is same with any other department. generate a recommendation to make the job responsibility distinct and effective."\
                "Make the recommendation per point in department that has same jobdesk and modified version of it."\
                "if it needed generate which job desk that need to cut or merge because of the job desk to differentiate and the reason behind it."\
                "Generate the output per point with department and recommended job responsibilities as keys."\
                "do not generate again the input job responsibility as an output."\
                "do not generate the answear in json."\
                "Also generate the reason behind every decision you made."\
                "[/INST]\n"\
                "Output:"
 #make a few shot
        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response


In [114]:
ans_tech = tech_optimized_org(job_tech)
print(ans_tech)



1. Department: Data
Recommended Job Responsibilities:
* Develop and maintain predictive models to solve business problems or requirements in e-commerce and digital marketing domains.
* Work with cross-functional teams to identify, analyze, and interpret trends or patterns in complex data sets.
* Build and maintain machine learning models to improve the accuracy of predictions and recommendations.
* Communicate insights and recommendations to stakeholders through clear and concise visualizations.
* Collaborate with software development teams to integrate predictive models into production systems.
* Stay up-to-date with the latest technology trends and best practices in data science and machine learning.

Reasoning:
The job responsibilities of the Data department are similar to those of the Software Quality department, specifically in terms of data analysis and machine learning. To differentiate the roles, the Data department should focus more on predictive modeling and data science, w

Sales Support

In [24]:
job_sales_support = [
  {
    "department": "Creative Designer",
    "JobResponsibilities": [
      "Plan, implement, and develop visual content aligning with the category's look and feel",
      "Stay updated on market trends for improved visual communication",
      "Coordinate with Content & Promotion teams to align visual communication with Work Order",
      "Collaborate with various teams for impactful promotions",
      "Evaluate and improve work processes regularly",
      "Establish action plans for efficient task completion",
      "Report job progress regularly"
    ]
  },
  {
    "department": "Head of Regional Sales",
    "JobResponsibilities": [
      "Develop guidance for ecommerce goals, quality, mix margin, and price",
      "Identify opportunities for revenue growth and collaborate with category teams",
      "Develop sales strategy in coordination with category teams",
      "Collaborate with various teams for new seller initiatives and promotions",
      "Work on special promotions and campaigns",
      "Collaborate with Marketing teams for performance ads"
    ]
  },
  {
    "department": "Head of Sales Excellence",
    "JobResponsibilities": [
      "Collaborate with category teams for seller development projects",
      "Work with Seller Activation & Engagement team for communication programs",
      "Develop understanding of seller platform tools and features",
      "Benchmark industry practices and collaborate with Operations for Seller Care",
      "Provide management updates on Seller Development",
      "Provide feedback and guidance to team members",
      "Plan and support team members' skill development"
    ]
  },
  {
    "department": "Head of Sales Support",
    "JobResponsibilities": [
      "Develop business flow, policies, and system improvements",
      "Strategize and execute marketing activities for merchant acquisition",
      "Build strategic partnerships with the government and associations",
      "Oversee fast-moving product acquisition and consignment",
      "Nurture Merchant Partners and ensure data validity",
      "Lead business process policies and compliance"
    ]
  },
  {
    "department": "Head of Sales",
    "JobResponsibilities": [
      "Develop and implement category merchandising plan",
      "Drive sales through quality, pricing, and product development expertise",
      "Identify opportunities for revenue growth",
      "Leverage content, data, and analytics for revenue growth",
      "Translate objectives into online features and content",
      "Maintain knowledge of competitor strategies",
      "Responsible for category profit and loss"
    ]
  },
  {
    "department": "Planner",
    "JobResponsibilities": [
      "Own the design and maintenance of metrics and reports",
      "Lead demand planners for inventory targets and sales",
      "Lead inventory management process improvement",
      "Provide insight into forecast error and drive continuous improvement",
      "Review historical sales trends and develop forecast models",
      "Establish action plans for efficient task completion"
    ]
  },
  {
    "department": "Promotion",
    "JobResponsibilities": [
      "Plan and develop content aligned with the category's look and feel",
      "Update market trends for promotion insights",
      "Coordinate with Merchandising and Brand Management for promotion concepts",
      "Evaluate promotion programs and provide feedback",
      "Create work orders for creative designers",
      "Establish action plans for efficient task completion"
    ]
  },
  {
    "department": "Sales Admin",
    "JobResponsibilities": [
      "Ensure accurate administrative and onboarding processes",
      "Monitor and assist in product upload for merchant partners",
      "Handle purchase order administration",
      "Analyze and investigate issues from merchant partners",
      "Take prompt action to achieve work goals",
      "Do all tasks assigned by the superior"
    ]
  },
  {
    "department": "Sales Analytics",
    "JobResponsibilities": [
      "Expert in advanced analytics techniques and programming languages",
      "Manage and execute analytics projects",
      "Build strategic relationships with technical partners",
      "Provide insights to optimize business pillars",
      "Develop team members"
    ]
  },
  {
    "department": "Sales Community",
    "JobResponsibilities": [
      "Create, plan, and propose community seller engagement programs",
      "Execute community engagement and activation programs",
      "Build partnerships with marketing and merchant partners",
      "Prepare communication collateral materials",
      "Generate improvement plans from feedback and analysis",
      "Establish action plans for efficient task completion"
    ]
  },
  {
    "department": "Sales Digital",
    "JobResponsibilities": [
      "Manage website Seller Center and track website performance",
      "Create and update website content, manage SEO and ads performance",
      "Compile and analyze social media metrics",
      "Maintain social media strategy and planning",
      "Create video content for social media",
      "Stay updated on social marketing trends"
    ]
  },
  {
    "department": "Sales Director",
    "JobResponsibilities": [
      "Optimize process flow and reduce variability",
      "Verify and validate products and processes",
      "Research and recommend new technologies",
      "Take prompt action to achieve work goals",
      "Conduct benchmarking and research for improvements",
      "Lead business projects planning and execution",
      "Develop cost-benefit/ROI analysis"
    ]
  },
  {
    "department": "Sales Marketplace",
    "JobResponsibilities": [
      "Develop product category and acquire new sellers",
      "Identify potential sellers and collaborate with Lead Acquisition",
      "Create market analysis and business plans",
      "Design and execute effective promotion plans",
      "Monitor and optimize seller operational and sales performance",
      "Maintain relationships with sellers",
      "Establish action plans for efficient task completion"
    ]
  },
  {
    "department": "Sales Trading",
    "JobResponsibilities": [
      "Identify product availability based on market trends",
      "Prospect local supplier sourcing for trading products",
      "Create business plans and promotions",
      "Liaise with internal teams for product availability and sales performance",
      "Maximize product profitability and negotiate with partners",
      "Maintain relationships with sellers",
      "Establish action plans for efficient task completion"
    ]
  },
  {
    "department": "Sales Trainer",
    "JobResponsibilities": [
      "Execute administrative training activities based on SLA",
      "Maintain merchant experience during onboarding",
      "Evaluate and improve work processes",
      "Report work implementation based on needs",
      "Responsible for tasks assigned by the superior"
    ]
  }
]


In [117]:
def sales_support_optimized_org(input_data, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 4000,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            #"Temperature" : 0.7
            # "stop_sequences": [""]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"generate the response with {goal} as consideration"\
                f"check in this data {job_sales_support} is there any job responsibility in one department that is same with any other department.\n"\
                "If there is job responsibility in one department that is same with any other department. generate a recommendation to make the job responsibility distinct and effective."\
                "Make the recommendation per point in department that has same jobdesk and modified version of it."\
                "if it needed generate which job desk that need to cut or merge because of the job desk to differentiate and the reason behind it."\
                "Generate the output per point with department and recommended job responsibilities as keys."\
                "do not generate again the input job responsibility as an output."\
                "[/INST]\n"\
                "Output:"
 #make a few shot
        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response


In [116]:
ans_sales_support = sales_support_optimized_org(job_sales_support)
print(ans_sales_support)



1. Department: Creative Designer
Recommended Job Responsibilities:
	* Develop visual content aligning with the category's look and feel.
	* Stay updated on market trends for improved visual communication.
	* Coordinate with Content & Promotion teams to align visual communication with Work Order.
	* Collaborate with various teams for impactful promotions.
	* Evaluate and improve work processes regularly.
	* Establish action plans for efficient task completion.
	* Report job progress regularly.

Modified Job Responsibilities:

* Design and maintain visual identity of the company, including logos, icons, and branding.
* Develop and implement design standards and guidelines for all visual content.
* Collaborate with Marketing teams to ensure consistent branding across all channels.
* Create and manage visual assets for website, social media, and marketing campaigns.
* Conduct market research to stay updated on industry trends and best practices.
* Provide training and support to team mem

Sales marketing

In [130]:
job_sales_marketing = [
    {
        "Title": "Activation",
        "JobResponsibilities": [
            "Create and execute social media campaign for each category or brand by coordinating with Trade Partnership and Brand Strategy Team",
            "Develop inspiring social media campaign which achieve corporate goals and suit with brand persona by coordinating with KOL and related other parties",
            "Set-up, optimize and maintain company pages on each social platform by generate, edit, publish and share daily content (text, images, video, etc.) which build meaningful connections with consumer audiences",
            "Create scheduling content for daily publish for FB, IG, TW and weekly for YT",
            "Create interactive live report event to published in social media platform to support event.",
            "Monitor and moderate user-generated content and respond in a timely and professional manner in keeping with company and brand value",
            "Compare and analyze posts to determine which ones are performing better and what steps can be taken to improve audience engagement for future posts",
            "Be aware of the latest trends in social marketing to keep the company on the forefront of any new emerging platforms in the social media space",
            "Launch marketing projects in support of brand initiatives, including planning, creative development, execution, and optimization, in partnership with internal marketing teams.",
            "Develop innovative marketing plans that generate engaging connections between product to customer with the objective of driving consumer excitement and motivating purchase",
            "Partner with internal marketing and creative teams to create and optimize content, develop any promotions in events and monitor and report on results to drive events traffic.",
            "Manage printing/execution for all event materials including evergreen collateral, signage, and educational content",
            "Generate and analyze merchandise reports and direct brand appropriate merchandising moves to maximize presentation and drive sales",
            "Manage budget and monitor spend against planned objectives"
        ]
    },
    {
        "Title": "Brand",
        "JobResponsibilities": [
            "Menyusun brand story dan membantu Brand Management dalam mengembangkan complete brand architecture.",
            "Melaksanakan dan memonitor kegiatan advertising (strategy/ creative brief development, copy evaluation, copy clearance, commercial production dan copy testing), kegiatan above-the-line, below-the-line consumer communication (print, outdoor, direct consumer marketing, public relations, point of sell materials, dll).",
            "Mengembangkan dan melaksanakan media strategist untuk mendapatkan target consumers termasuk level, mix dan market prioritization).",
            "Memonitor, mengevaluasi dan menganalisa consumer / customer/ market trends bersama dengan tim Research & Analytics untuk mengidentifikasi business drivers dan opportunities, termasuk dalam memberikan feedback kepada  tim Research & Analytics.",
            "Melakukan kerjasama dan koordinasi dengan tim lain untuk memastikan kelancaran pelaksanaan multi functional projects (inisiatif/ promosi/ pricing plan development, dll.",
            "Mengevaluasi dan melakukan improvement secara berkelanjutan terhadap proses kerja yang dilakukan di bagaiannya."
        ]
    },
    {
        "Title": "Campaign",
        "JobResponsibilities": [
            "Create campaign concept, strategy and communicate campaign brief for more than 1 campaign project.",
            "Providing campaign insights based on campaign performance and historical data",
            "Create and standardized campaign brief",
            "Collect and share campaign best practices across categories: landing page, visual asset, communication plan",
            "Seek deals, Review the brief and approve the promotion budget for each campaign",
            "Owner of abc Homepage and Pricing Tools",
            "Main campaign initiator of abc Mega Campaign and Flash Sale",
            "Work with product and tech to deliver great customer experience when looking for promotion and seller experience in creating promotion",
            "Work within the available resources (promotion & marketing budget) to achieve the required target"
        ]
    },
    {
        "Title": "Creative Designer",
        "JobResponsibilities": [
            "Plan, implement and develop visual look of the content, products, and promotion program within the category to ensure those parts align with looks & feels of the category",
            "Update the latest trend on the market to gain insight that can improve visual communication on the design",
            "Maintain communication and coordination with Content & Promotion to ensure the visual communication aligns with the Work Order (WO)",
            "Collaborate with Content & Promotion, Merchandising Manager, VP of Category, and SVP of Category to create an impactful promo",
            "Evaluate and do regular improvement on the work process",
            "Establish an action plan for self and others to complete work efficiently and on time by setting priorities",
            "Fulfill all the task that is assigned by the superior",
            "Report the progress of the job regularly"
        ]
    },
    {
        "Title": "Head of Marketing",
        "JobResponsibilities": [
            "Developing and implementing marketing plans that are aligned with other Department strategy.",
            "Overseeing and managing PR, communication, social media and digital content strategy (not limited to be involved directly if required).",
            "Leading the strategy to grow the brand by building brand awareness, brand consideration and brand purchase intention.",
            "Oversee market research, pricing, product/service marketing, marketing communications, advertising and public relations.",
            "Optimising the marketing mix and touch point strategy through digital, and other off-line media when needed.",
            "Driving branding innovation to meet the business needs, and actively involved in growing new targeted customers.",
            "Developing solid brand, and campaign strategies to achieve business objective.",
            "Continually review changes to the market, consumer trends and the activities of competitors, adjusting the marketing plan if necessary.",
            "Lead employees to encourage maximum performance and dedication"
        ]
    },
    {
        "Title": "Partnership",
        "JobResponsibilities": [
            "Act as the primary contact of partners for needs related to marketing",
            "Building mutually beneficial cooperative relationships with third parties",
            "Exploring partner needs and formulating cooperation proposals according to the partner's brief",
            "Ensure all cooperation programs with partners run on time, execute flawlessly, and achieve the campaign targets that have been set together",
            "Monitor sales performance (sales volume) from good partners based on campaign",
            "Building good coordination and cooperation with related teams both internally and externally starting from the planning stage to the implementation and evaluation of each marketing partnership program that is carried out.",
            "Fully responsible for partner satisfaction with abc.com as a whole",
            "Responsible for maintaining abc.com reputation in front of partners"
        ]
    },
    {
        "Title": "Performance",
        "JobResponsibilities": [
            "Develop Digital Marketing Strategy, Tactical Planning, and its measurement based on the developed and established guidance by Brand Management.",
            "Compile and develop homepage content and thematic brand/thematic cross-category campaigns, along with their implementation, in coordination with the Category General Manager, Content & Promotion Strategist, and Campaign Management Staff.",
            "Create Marketing Programs related to the Digital Marketing platform and ensure their implementation, monitoring, and evaluation.",
            "Provide feedback for the development of an integrated Marketing Plan to the Brand Management Manager and ensure its execution in coordination with the Brand Management Manager and Marketing Communication Manager.",
            "Develop SEM & SEO strategies to ensure maximum demand generation and lead generation, consistent traffic growth, optimize sales conversions within the allocated budget, and ensure their implementation and evaluation.",
            "Analyze site analytics to evaluate the effectiveness and optimize online marketing programs, provide insights on best practices, develop improvement measures based on those insights, and maximize return on marketing investments.",
            "Continuously evaluate and improve the working processes within the respective area.",
            "Foster the development of each individual within the team.",
            "Collaborate with the Web Analyst to analyze and optimize performance based on the expected outcomes."
        ]
    }
]

In [133]:
def sales_marketing_optimized_org(input_data, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 3000,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            #"Temperature" : 0.7
            # "stop_sequences": [""]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"generate the response with {goal} as consideration"\
                f"check in this data {job_sales_marketing} is there any job responsibility in one department that is same with any other department.\n"\
                "If there is job responsibility in one department that is same with any other department. generate a recommendation to make the job responsibility distinct and effective."\
                "Make the recommendation per point in department that has same jobdesk and modified version of it."\
                "if it needed generate which job desk that need to cut or merge because of the job desk to differentiate and the reason behind it."\
                "Generate the output per point with department and recommended job responsibilities as keys."\
                "do not generate again the input job responsibility as an output."\
                "do not generate the answear in json."\
                "Also generate the reason behind every decision you made."\
                "Generate recommendation in the end."\
                "[/INST]\n"\
                "Output:"
 #make a few shot
        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response


In [134]:
ans_sales_marketing= sales_marketing_optimized_org(job_sales_marketing)
print(ans_sales_marketing)



1. Activation
Recommended Job Responsibilities:
* Develop and execute social media campaigns for each category or brand in coordination with Trade Partnership and Brand Strategy Team.
* Create inspiring social media campaigns that achieve corporate goals and suit brand persona in coordination with KOL and related parties.
* Set up, optimize, and maintain company pages on each social platform, generating daily content that builds meaningful connections with consumer audiences.
* Create scheduling content for daily publishing on FB, IG, TW, and weekly publishing on YT.
* Monitor and moderate user-generated content, responding promptly and professionally while maintaining company and brand values.
* Analyze posts to determine which ones perform better and improve audience engagement for future posts.
* Stay aware of the latest trends in social marketing, keeping the company at the forefront of emerging platforms in the social media space.

Reason: The job responsibilities of the Activat

Sales Platform 

In [125]:
job_sales_platform = [
    {
        "Title": "Platform Designer",
        "JobResponsibilities": [
            "Ensure a cohesive modern aesthetic and clean look/feel in all design elements affecting our products while ensuring their brand attributes are consistently represented.",
            "Translate UX Lo-Fi, user-flow/scenario & cases into a usable high-fidelity concepts and design in a creative\u00a0and visual way that engages its audience.",
            "Collaborate with Product Managers, Engineers, Researchers, and Designers team members on interaction design framework and execution to ensure\u00a0seamless transition between user experience design and visual design.",
            "Provide rationalize and justify design work and advocate for users\u2019 needs by communicate design intent to both internal and external teams.",
            "Proposed visual design concepts using variety of methods, including but not limited to prototypes to static screens with flow.",
            "Establish and maintain best practices and design methods for art direction and stay abreast of rich media\u00a0techniques, emerging technologies, research future trends and competitor strategy.",
            "Participate in user experience research and usability studies.",
            "Work across various lifecycle stages of a product, from new products to continuous product improvements through agile (scrum) process",
            "Mentor team Jr.Designer providing feedback, knowledge transfer, & advocates for general UI best practices"
        ]
    },
    {
        "Title": "Platform Engineer",
        "JobResponsibilities": [
            "Design and develop user interfaces that enhance the overall user experience.",
            "Collaborate with cross-functional teams to understand user requirements and translate them into intuitive and visually appealing interface designs.",
            "Utilize software technologies such as JavaScript, Flash, XML, HTML, and CSS to implement UX designs.",
            "Document UX designs, guidelines, and best practices for reference and future use",
            "Conduct usability testing sessions and user research activities to gather feedback and iterate on design solutions.",
            "Analyze and interpret analytics data to identify areas for UX enhancement and optimize user interactions.",
            "Stay up-to-date with emerging UX design trends, technologies, and industry best practices.",
            "Collaborate with developers, designers, and product managers to ensure seamless integration of the UX with the overall application."
        ]
    },
    {
        "Title": "Product Manager",
        "JobResponsibilities": [
            "Acts as the subject matter expert of the product",
            "Conduct extensive competitive product evaluations and analysis",
            "Evangelize ideas that solve real customer problems",
            "Gather user/ customers feedback to deeply understand their needs",
            "Collaborate with technology, trade partnership, operation, and marketing to craft solutions to customer needs",
            "Rapidly formulate concrete features that address needs and requirements expressed by internal users and customers",
            "Write clear, concise product requirements and user stories",
            "Document product functionality and communicate to users",
            "Review user stories to ensure that the delivery is the same as initial requirements",
            "Review scenario tests for the improvements, features, and projects to ensure that the delivery is the same as initial requirements",
            "Attend & lead agile product development process including backlog grooming, sprint planning and prioritization sessions and daily stand-ups"
        ]
    },
    {
        "Title": "Researcher",
        "JobResponsibilities": [
            "Observe and analyze user behavior to provide actionable insights to help develop abc's customer experience",
            "Conduct generative and foundational user research that will serve as the basis for new ideas and concepts, product roadmap, discovery, and initiative backlog to develop product and feature improvements",
            "Synthesize qualitative and quantitative data to provide key business and customer insights to product teams",
            "Collaborate with designer, product manager and other parties that related to construct design prototypes, evaluate, and incorporate results directly into design iterations and product decision-making",
            "Serve as a consultant to UX, Product Manager and other key stakeholders and teams",
            "Establish and maintain best practices, methods stay abreast of emerging technologies, research future trends and competitor strategy.",
            "Create new and adapt existing research methods and practices to deliver customer knowledge and advocate research findings to diverse audiences through written reports and oral presentations",
            "Monitor and track user issues to UX and product team and management",
            "Mentor team Associate UX Researcher to providing feedback, knowledge transfer, & advocates for general UX Research best practices --> For Sr. UX Researcher"
        ]
    }
]

In [131]:
def sales_platform_optimized_org(input_data, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 3000,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            #"Temperature" : 0.7
            # "stop_sequences": [""]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"generate the response with {goal} as consideration"\
                f"check in this data {job_sales_platform} is there any job responsibility in one department that is same with any other department.\n"\
                "If there is job responsibility in one department that is same with any other department. generate a recommendation to make the job responsibility distinct and effective."\
                "Make the recommendation per point in department that has same jobdesk and modified version of it."\
                "if it needed generate which job desk that need to cut or merge because of the job desk to differentiate and the reason behind it."\
                "Generate the output per point with department and recommended job responsibilities as keys."\
                "do not generate again the input job responsibility as an output."\
                "do not generate the answear in json."\
                "Also generate the reason behind every decision you made."\
                "[/INST]\n"\
                "Output:"
 #make a few shot
        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response


In [132]:
ans_sales_platform = sales_platform_optimized_org(job_sales_platform)
print(ans_sales_platform)



1. Department: Platform Designer
Recommended Job Responsibilities:
* Ensure a cohesive modern aesthetic and clean look/feel in all design elements affecting our products while ensuring their brand attributes are consistently represented.
* Translate UX Lo-Fi, user-flow/scenario & cases into a usable high-fidelity concepts and design in a creative and visual way that engages its audience.
* Collaborate with Product Managers, Engineers, Researchers, and Designers team members on interaction design framework and execution to ensure seamless transition between user experience design and visual design.
* Provide rationalize and justify design work and advocate for users’ needs by communicating design intent to both internal and external teams.
* Proposed visual design concepts using variety of methods, including but not limited to prototypes to static screens with flow.
* Establish and maintain best practices and design methods for art direction and stay abreast of rich media techniques, 

Skill Map 


In [6]:
job_input =  {
  "departments": [
    {
      "name": "Platform Designer",
      "recommendedJobResponsibilities": [
        "Ensure a cohesive modern aesthetic and clean look/feel in all design elements affecting our products while ensuring their brand attributes are consistently represented.",
        "Translate UX Lo-Fi, user-flow/scenario & cases into a usable high-fidelity concepts and design in a creative and visual way that engages its audience.",
        "Collaborate with Product Managers, Engineers, Researchers, and Designers team members on interaction design framework and execution to ensure seamless transition between user experience design and visual design.",
        "Provide rationalize and justify design work and advocate for users’ needs by communicating design intent to both internal and external teams.",
        "Proposed visual design concepts using variety of methods, including but not limited to prototypes to static screens with flow.",
        "Establish and maintain best practices and design methods for art direction and stay abreast of rich media techniques, emerging technologies, research future trends and competitor strategy.",
        "Participate in user experience research and usability studies.",
        "Work across various lifecycle stages of a product, from new products to continuous product improvements through agile (scrum) process."
      ],
      "reasoning": "The job responsibilities of the Platform Designer and Platform Engineer have some similarities, such as designing user interfaces and collaborating with cross-functional teams. However, the Platform Designer focuses more on the visual design and user experience, while the Platform Engineer focuses more on the technical implementation of the design."
    },
    {
      "name": "Platform Engineer",
      "recommendedJobResponsibilities": [
        "Design and develop user interfaces that enhance the overall user experience.",
        "Collaborate with cross-functional teams to understand user requirements and translate them into intuitive and visually appealing interface designs.",
        "Utilize software technologies such as JavaScript, Flash, XML, HTML, and CSS to implement UX designs.",
        "Document UX designs, guidelines, and best practices for reference and future use.",
        "Conduct usability testing sessions and user research activities to gather feedback and iterate on design solutions.",
        "Analyze and interpret analytics data to identify areas for UX enhancement and optimize user interactions.",
        "Stay up-to-date with emerging UX design trends, technologies, and industry best practices.",
        "Collaborate with developers, designers, and product managers to ensure seamless integration of the UX with the overall application."
      ],
      "reasoning": "The job responsibilities of the Platform Engineer and the Product Manager have some similarities, such as collaborating with cross-functional teams and ensuring seamless integration of the UX with the overall application. However, the Platform Engineer focuses more on the technical implementation of the design, while the Product Manager focuses more on the overall product strategy and user needs."
    },
    {
      "name": "Product Manager",
      "recommendedJobResponsibilities": [
        "Acts as the subject matter expert of the product.",
        "Conduct extensive competitive product evaluations and analysis.",
        "Evangelize ideas that solve real customer problems.",
        "Gather user/customers feedback to deeply understand their needs.",
        "Collaborate with technology, trade partnership, operation, and marketing to craft solutions to customer needs.",
        "Rapidly formulate concrete features that address needs and requirements expressed by internal users and customers.",
        "Write clear, concise product requirements and user stories.",
        "Document product functionality and communicate to users.",
        "Review user stories to ensure that the delivery is the same as initial requirements.",
        "Review scenario tests for the improvements, features, and projects to ensure that the delivery is the same as initial requirements.",
        "Attend & lead agile product development process including backlog grooming, sprint planning and prioritization sessions and daily stand-ups."
      ],
      "reasoning": "The job responsibilities of the Product Manager and the Researcher have some similarities, such as gathering user feedback and conducting research. However, the Product Manager focuses more on the overall product strategy and user needs, while the Researcher focuses more on conducting research to provide actionable insights to help develop abc's customer experience."
    },
    {
      "name": "Researcher",
      "recommendedJobResponsibilities": [
        "Observe and analyze user behavior to provide actionable insights to help develop abc's customer experience.",
        "Conduct generative and foundational user research that will serve as the basis for new ideas and concepts, product roadmap, discovery, and initiative backlog to develop product and feature improvements.",
        "Synthesize qualitative and quantitative data to provide key business and customer insights to product teams.",
        "Collaborate with designer, product manager and other parties that related to construct design prototypes, evaluate, and incorporate results directly into design iterations and product decision-making.",
        "Serve as a consultant to UX, Product Manager and other key stakeholders and teams.",
        "Establish and maintain best practices, methods stay abreast of emerging technologies, research future trends and competitor strategy.",
        "Create new and adapt existing research methods and practices to deliver customer knowledge and advocate research findings to diverse audiences through written reports and oral presentations.",
        "Monitor and track user issues to UX and product team and management."
      ],
      "reasoning": "The job responsibilities of the Researcher and the UX Researcher have some similarities, such as conducting research and providing actionable insights. However, the Researcher focuses more on observing and analyzing user behavior, while the UX Researcher focuses more on conducting research to provide actionable insights to help develop abc's customer experience."
    },
    {
      "name": "UX Researcher",
      "recommendedJobResponsibilities": [
        "Conduct generative and foundational user research that will serve as the basis for new ideas and concepts, product roadmap, discovery, and initiative backlog to develop product and feature improvements.",
        "Synthesize qualitative and quantitative data to provide key business and customer insights to product teams.",
        "Collaborate with designer, product manager and other parties that related to construct design prototypes, evaluate, and incorporate results directly into design iterations and product decision-making.",
        "Serve as a consultant to UX, Product Manager and other key stakeholders and teams.",
        "Establish and maintain best practices, methods stay abreast of emerging technologies, research future trends and competitor strategy.",
        "Create new and adapt existing research methods and practices to deliver customer knowledge and advocate research findings to diverse audiences through written reports and oral presentations.",
        "Monitor and track user issues to UX and product team and management."
      ],
      "reasoning": "The job responsibilities of the UX Researcher and the Researcher have some similarities, such as conducting research and providing actionable insights. However, the UX Researcher focuses more on conducting research to provide actionable insights to help develop abc's customer experience, while the Researcher focuses more on observing and analyzing user behavior."
    }
  ],
  "recommendation": [
    "Ensure a cohesive modern aesthetic and clean look/feel in all design elements affecting our products while ensuring their brand attributes are consistently represented.",
    "Design and develop user interfaces that enhance the overall user experience.",
    "Act as the subject matter expert of the product and gather user/customers feedback to deeply understand their needs.",
    "Observe and analyze user behavior to provide actionable insights to help develop abc's customer experience.",
    "Conduct generative and foundational user research that will serve as the basis for new ideas and concepts, product roadmap, discovery, and initiative backlog to develop product and feature improvements."
  ]
}

In [10]:
def skill_mapping(job_input,goal_input, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 3000,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            #"Temperature" : 0.7
            "stop_sequences": ["}\n\n"]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"generate the response with {goal_input} as consideration"\
                f"generate skill set that is necessary for department in {job_input}."\
                "do not generate again the input job responsibility as an output."\
                "generate the answear in json with key : department, skill set and the reason behind it for every department reason."\
                "do not generate any reasoning from data provided"\
                "[/INST]\n"\
                "Output:"
 #make a few shot
        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response
    #make sure put the goal first in the function skill(goal,job_input) 

ans_skill_ux = skill_mapping(goal, job_input)

In [12]:
print(ans_skill_ux)


{
"departments": [
{
"name": "Platform Designer",
"skillSet": [
"Visual design skills",
"User experience (UX) design skills",
"Interaction design skills",
"Prototyping skills",
"Design software skills (e.g. Sketch, Figma, Adobe XD)"
],
"reasoning": "The Platform Designer is responsible for ensuring a cohesive modern aesthetic and clean look/feel in all design elements affecting abc's products. They must have strong visual design skills, user experience (UX) design skills, interaction design skills, prototyping skills, and proficiency in design software such as Sketch, Figma, or Adobe XD to create high-quality design concepts that align with abc's brand attributes and enhance the overall user experience."
},
{
"name": "Platform Engineer",
"skillSet": [
"Software engineering skills",
"Front-end development skills (e.g. JavaScript, HTML, CSS)",
"Back-end development skills (e.g. Node.js, Ruby on Rails)",
"Database management skills",
"API integration skills"
],
"reasoning": "The Platform